In [70]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import numpy as np
import contextily as ctx

In [2]:
taz = gpd.read_file(r"D:\GitHub\TDM-Documentation\v9x\v910\whats-new\data\taz\WFv900_TAZ.shp")[['TAZID','CO_NAME','geometry']]
bd = pd.read_csv(r"D:\GitHub\TDM-Documentation\v9x\v910\whats-new\data\BaseDistribution.csv")[[';Z','Airport','PVU']].rename(columns={';Z':'TAZID'})

tazbd = gpd.GeoDataFrame(pd.merge(bd,taz,on='TAZID',how='left'))

In [127]:
def plot_pvu(gdf,county):

    # Fix CRS for basemap
    if gdf.crs is None:
        gdf.set_crs(epsg=26912, inplace=True)
    if gdf.crs.to_string() != 'EPSG:3857':
        gdf = gdf.to_crs(epsg=3857)
        
    # set color bins for legend
    bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 40, 60, 80, 100]
    cmap = plt.cm.get_cmap('YlOrRd', len(bins) - 2)
    colors_list = ['#00000000'] + [cmap(i) for i in range(cmap.N)] 
    cmap_with_gray = colors.ListedColormap(colors_list)
    norm = colors.BoundaryNorm(bins, cmap_with_gray.N)
    
    # Plot the data
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    # Plot the TAZ polygons and color them by PVU using the new colormap
    gdf.plot(column='PVU', cmap=cmap_with_gray, norm=norm, linewidth=0.05, ax=ax, edgecolor='black', legend=False)
    centroid = gdf[gdf['TAZID'] == 2946].geometry.centroid
    ax.scatter(centroid.x, centroid.y, color='blue', marker='*', s=100, edgecolor='blue', zorder=3)  # s controls dot size
    
    # Create the color bar with discrete intervals
    sm = plt.cm.ScalarMappable(cmap=cmap_with_gray, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, boundaries=bins, ticks=bins, shrink=0.5, aspect=12)
    cbar.set_label('Number of Trips')

    # Add basemap using contextily with OpenStreetMap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, alpha=.6)
    
    # PVU Legend
    blue_patch = plt.Line2D([0], [0], marker='*', color='w', label='PVU', markerfacecolor='blue', markersize=15)
    ax.legend(handles=[blue_patch], loc='upper right', bbox_to_anchor=(1.22, 0.235)) 
    
    # Set axis limits to zoom (remove 30% from top and right)
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Reduce by 30% on the right (x-axis) and 30% on the top (y-axis)
    new_xlim = (xlim[0] + 0.18 * (xlim[1] - xlim[0]), xlim[0] + 0.6 * (xlim[1] - xlim[0]))
    new_ylim = (ylim[0] + 0.15 * (ylim[1] - ylim[0]), ylim[0] + 0.58 * (ylim[1] - ylim[0]))
    
    # Set new axis limits
    ax.set_xlim(new_xlim)
    ax.set_ylim(new_ylim)

    # Set plot title and labels and plot
    ax.set_axis_off()
    
    plt.rcParams["figure.figsize"]=6,12
    plt.tight_layout()
    plt.savefig(f'D:\\GitHub\\TDM-Documentation\\v9x\\v910\\whats-new\\_pictures\\pvu.png', bbox_inches='tight', dp=12000)
    plt.close(fig)

    plt.show


In [128]:
plot_pvu(tazbd,'ALL')
#plot_pvu(tazbd,'UTAH')
#plot_pvu(tazbd,'SALT LAKE')

C:\Users\cday\AppData\Local\Temp\ipykernel_24736\3204087333.py:54: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "dp" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig(f'D:\\GitHub\\TDM-Documentation\\v9x\\v910\\whats-new\\_pictures\\pvu.png', bbox_inches='tight', dp=12000)
